In [2]:
import gurobipy as grb
from gurobipy import GRB
import networkx as nx

In [3]:
import json

space_age = json.load(open("data/space-age.json"))



In [4]:
G = nx.DiGraph()
from collections import defaultdict


machines = defaultdict(set)
for n, f in space_age["furnace"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("furnace", n))

for n, f in space_age["assembling-machine"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("assembling-machine", n))


modules = defaultdict(dict)
for n, m in space_age["module"].items():
    modules[m["category"]][m["name"]] = m["effect"]

print(modules)

max_quality_level = 0
for n, q in space_age["quality"].items():
    max_quality_level = max(max_quality_level, q["level"])

items = set()

for n, r in space_age["recipe"].items():
    if "ingredients" in r:
        items.update((i["type"], i["name"]) for i in r["ingredients"])
    if "results" in r:
        items.update((i["type"], i["name"]) for i in r["results"])


defaultdict(<class 'dict'>, {'speed': {'speed-module': {'speed': 0.2, 'consumption': 0.5, 'quality': -0.1}, 'speed-module-2': {'speed': 0.3, 'consumption': 0.6, 'quality': -0.15}, 'speed-module-3': {'speed': 0.5, 'consumption': 0.7, 'quality': -0.25}}, 'efficiency': {'efficiency-module': {'consumption': -0.3}, 'efficiency-module-2': {'consumption': -0.4}, 'efficiency-module-3': {'consumption': -0.5}}, 'productivity': {'productivity-module': {'productivity': 0.04, 'consumption': 0.4, 'pollution': 0.05, 'speed': -0.05}, 'productivity-module-2': {'productivity': 0.06, 'consumption': 0.6, 'pollution': 0.07, 'speed': -0.1}, 'productivity-module-3': {'productivity': 0.1, 'consumption': 0.8, 'pollution': 0.1, 'speed': -0.15}}, 'quality': {'quality-module': {'quality': 0.1, 'speed': -0.05}, 'quality-module-2': {'quality': 0.2, 'speed': -0.05}, 'quality-module-3': {'quality': 0.25, 'speed': -0.05}}})


In [5]:

from itertools import chain, combinations_with_replacement

all_crafting_variables = set()
all_item_variables = set()

ingredients_to_recipes = defaultdict(set)
products_to_recipes = defaultdict(set)
for _, r in space_age["recipe"].items():

    allow_quality = int(r.get("allow_quality", True))
    for q in range(allow_quality * (max_quality_level+1)):

        for i in r.get("ingredients", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))
        for i in r.get("results", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))


        for m in machines[r.get("category", None)]:
            machine = space_age[m[0]][m[1]]

            slots = machine.get("module_slots", 0)
            possible_modules_categories = machine.get("allowed_module_categories", modules.keys())
            possible_modules = set().union(*(set(modules[c].keys()) for c in possible_modules_categories))

            possible_modules = {m for m in possible_modules if space_age["module"][m].get("tier", 1) == 2}

        
            for chosen_modules in chain.from_iterable(combinations_with_replacement(possible_modules, i) for i in range(slots)):
                all_crafting_variables.add((r["name"], q, m, chosen_modules))

                for k, i in enumerate(r.get("ingredients", [])):
                    ingredients_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add(((r["name"], q, m, chosen_modules), k))
                for k, i in enumerate(r.get("results", [])):
                    products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add(((r["name"], q, m, chosen_modules), k))


In [ ]:
model = grb.Model()

total_machines = model.addVars(all_crafting_variables, obj=1, vtype=GRB.INTEGER)
machine_usage = model.addVars(all_crafting_variables)
used = model.addVars(all_item_variables, name="used")
produced = model.addVars(all_item_variables, obj=1, name="produced")

used["item", "scrap", 0].setAttr(GRB.Attr.LB, -float("inf"))
produced["item", "quality-module-3", 5].setAttr(GRB.Attr.LB, 1)

model.addConstrs(used[i] <= produced[i] for i in all_item_variables)
model.addConstrs(machine_usage[i] <= total_machines[i] for i in all_crafting_variables)

for i in all_item_variables:
    expr_used = grb.LinExpr()
    for (rname, q, m, mods), k in ingredients_to_recipes.get(i, []):
        r = space_age["recipe"][rname]
        ing = r.get("ingredients", [])[k]
        amt = ing.get("amount", 0)
        if amt:
            expr_used.addTerms(amt, machine_usage[(rname, q, m, mods)])
    model.addConstr(used[i] == expr_used)

    for (r, q, m, modules), k in products_to_recipes[i]:
        pass

model.optimize()

Set parameter Username
Set parameter LicenseID to value 2732003
Academic license - for non-commercial use only - expires 2026-11-03


AttributeError: 'list' object has no attribute 'setAttr'

: 